<a href="https://colab.research.google.com/github/Loryana/OCProjet7/blob/main/Projet_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow --quiet
!pip install pyngrok --quiet
!pip install dagshub --quiet
import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.0/251.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/1

In [2]:
!mlflow --version

mlflow, version 2.17.0


In [3]:
import dagshub
dagshub.init(repo_owner='loryana9', repo_name='OCProjet7', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=112fbaff-222d-436f-bdff-62405ef35096&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=49bddc6a07cf386212c3bdd8fbc94d962fe4e6747249f961739bab5798fc468c




Output()

Accessing as loryana9

Initialized MLflow to track repo "loryana9/OCProjet7"

Repository loryana9/OCProjet7 initialized!

In [4]:
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [5]:
# Set the tracking URI.
mlflow.set_tracking_uri("https://dagshub.com/loryana9/OCProjet7.mlflow")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

2024/10/22 13:23:56 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart' does not exist. Creating a new experiment.
Successfully registered model 'tracking-quickstart'.
2024/10/22 13:24:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1
Created version '1' of model 'tracking-quickstart'.


2024/10/22 13:24:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-snail-995 at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/0/runs/35560f3b9e7d401d833a36683e5b5d6a.
2024/10/22 13:24:11 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/0.


In [6]:
# Load the model back for predictions as a generic Python Function model
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


In [7]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [8]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [9]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [10]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [11]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [12]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/10/22 13:24:17 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 33s 735ms/step - loss: 35.0729 - root_mean_squared_error: 5.9222
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.6868 - root_mean_squared_error: 5.8038   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 33.4984 - root_mean_squared_error: 5.7876 - val_loss: 31.5978 - val_root_mean_squared_error: 5.6212

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 11s 254ms/step - loss: 30.7925 - root_mean_squared_error: 5.5491
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.9169 - root_mean_squared_error: 5.5603   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.8227 - root_mean_squared_error: 5.5518 - val_loss: 29.2345 - val_root_mean_squared_error: 5.4069

Epoch 3/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 29.7095 - root_mean_squared_error: 5.4506
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29.0626 - root_mean_squared_error: 5.3908 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.9081 - root_mean_squared_error: 5.3764 - val_loss: 27.0465 - val_root_mea

2024/10/22 13:24:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run clean-shrew-782 at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1/runs/0af8260665a54787ab6ad1737e0ac22a.

2024/10/22 13:24:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1.



Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 36.8111 - root_mean_squared_error: 6.0672
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9844 - root_mean_squared_error: 2.2647 
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.1787 - root_mean_squared_error: 2.0982 - val_loss: 0.7149 - val_root_mean_squared_error: 0.8455

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 345ms/step - loss: 0.6265 - root_mean_squared_error: 0.7915
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6891 - root_mean_squared_error: 0.8300   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6789 - root_mean_squared_error: 0.8238 - val_loss: 0.7455 - val_root_mean_squared_error: 0.8634

Epoch 3/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 170ms/step - loss: 0.8033 - root_mean_squared_error: 0.8963
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5934 - root_mean_squared_error: 0.7700  
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5943 - root_mean_squared_error: 0.7707 - val_loss: 0.5692 - val_root_mean_squared_erro

2024/10/22 13:24:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run stately-bee-929 at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1/runs/ce01fe32cd4d456dbd37e23c59522ab2.

2024/10/22 13:24:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1.



...

Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 23s 533ms/step - loss: 36.6719 - root_mean_squared_error: 6.0557
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.1062 - root_mean_squared_error: 5.7518   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 31.8241 - root_mean_squared_error: 5.6370 - val_loss: 20.2456 - val_root_mean_squared_error: 4.4995

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 348ms/step - loss: 20.6623 - root_mean_squared_error: 4.5456
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18.3066 - root_mean_squared_error: 4.2758   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.6583 - root_mean_squared_error: 4.1975 - val_loss: 10.7538 - val_root_mean_squared_error: 3.2793

Epoch 3/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 180ms/step - loss: 10.2794 - root_mean_squared_error: 3.2061
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5402 - root_mean_squared_error: 3.0873   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.1609 - root_mean_squared_error: 3.0241 - val_loss: 5.7477 - val_root_mea

2024/10/22 13:26:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run abrasive-gnu-865 at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1/runs/ac4d5337947c44a99e28c4d125ebf694.

2024/10/22 13:26:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1.



100%|██████████| 8/8 [01:46<00:00, 13.33s/trial, best loss: 0.727226197719574]


2024/10/22 13:26:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run trusting-hog-571 at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1/runs/c84cefe83a9a419f92e6aec2bdde2873.
2024/10/22 13:26:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/OCProjet7.mlflow/#/experiments/1.


Best parameters: {'lr': 0.03948247970904209, 'momentum': 0.27478127291411103}
Best eval rmse: 0.727226197719574


In [14]:
# Load the model back for predictions as a generic Python Function model
mlflow.pyfunc.load_model( "models:/wine-quality/1")

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.tensorflow
  run_id: 7e39b0e29d064d91afdccf652387b0c5